In [1]:
from sympy import *
import numpy as np
from scipy.spatial.transform import Rotation

In [2]:
# -- States --

# Quaternion
qw, qx, qy, qz = symbols(r"q_w, q_x, q_y, q_z")
quaternion = Matrix([qw, qx, qy, qz])

# Bias in gyro reading
bx_g, by_g, bz_g = symbols(r'b_{x_g}, b_{y_g}, b_{z_g}')
bias_gyro = Matrix([bx_g, by_g, bz_g]) 

# Bias in accelerometer reading
bx_a, by_a, bz_a = symbols(r'b_{x_a}, b_{y_a}, b_{z_a}')
bias_accelerometer = Matrix([bx_a, by_a, bz_a])

# Bias in magnetometer reading
bx_m, by_m, bz_m = symbols(r'b_{x_m}, b_{y_m}, b_{z_m}')
bias_magnetometer = Matrix([bx_m, by_m, bz_m])

states_s = Matrix([quaternion, bias_gyro, bias_magnetometer])

In [3]:
# -- Inputs --
wx, wy, wz = symbols(r"w_x, w_y, w_z")
gyro_reading = Matrix([wx, wy, wz])

inputs_u = [gyro_reading]

In [4]:
# -- Measurements --
ax, ay, az = symbols(r"a_x, a_y, a_z")
accelerometer_reading = Matrix([ax, ay, az])

In [5]:
# -- Parameters --
dt = Symbol(r"\Delta t") # Timestep, s

g = symbols("g")
gravity_vec = Matrix([0, 0, g]) # Acceleration due to gravity in world frame, 3x m/s^2

mag_x, mag_y, mag_z = symbols(r"m_x, m_y, m_z")
mag_local = Matrix([mag_x, mag_y, mag_z]) # Local magnetic field vector, 3x Gauss

parameters = [dt, g, mag_local]

In [6]:
def rot_mat_from_quat(q):
    rotation_matrix = Matrix([
        [1 - 2*q[2]**2 - 2*q[3]**2, 2*q[1]*q[2] - 2*q[3]*q[0], 2*q[1]*q[3] + 2*q[2]*q[0]],
        [2*q[1]*q[2] + 2*q[3]*q[0], 1 - 2*q[1]**2 - 2*q[3]**2, 2*q[2]*q[3] - 2*q[1]*q[0]],
        [2*q[1]*q[3] - 2*q[2]*q[0], 2*q[2]*q[3] + 2*q[1]*q[0], 1 - 2*q[1]**2 - 2*q[2]**2]
    ])
    return rotation_matrix

In [7]:
def omega_skew_matrix(omega):
    w1, w2, w3 = omega
    mat = Matrix([
        [0, -w1, -w2, -w3],
        [w1, 0, w3, -w2],
        [w2, -w3, 0, w1],
        [w3, w2, -w1, 0]
    ])
    return mat

The state transition function ($f$) is defined as follows:
$$
s_{k+1} = f(s_k, u_k)
$$

In [8]:
state_transition_function_f = Matrix([
    quaternion + 1/2 * omega_skew_matrix(gyro_reading - bias_gyro) * quaternion * dt, 
    # world frame q_next
    bias_gyro, # assume bias evolves randomly
    bias_magnetometer, # assume bias evolves randomly
])

In [9]:
print(state_transition_function_f)

Matrix([[\Delta t*(q_x*(0.5*b_{x_g} - 0.5*w_x) + q_y*(0.5*b_{y_g} - 0.5*w_y) + q_z*(0.5*b_{z_g} - 0.5*w_z)) + q_w], [\Delta t*(q_w*(-0.5*b_{x_g} + 0.5*w_x) + q_y*(-0.5*b_{z_g} + 0.5*w_z) + q_z*(0.5*b_{y_g} - 0.5*w_y)) + q_x], [\Delta t*(q_w*(-0.5*b_{y_g} + 0.5*w_y) + q_x*(0.5*b_{z_g} - 0.5*w_z) + q_z*(-0.5*b_{x_g} + 0.5*w_x)) + q_y], [\Delta t*(q_w*(-0.5*b_{z_g} + 0.5*w_z) + q_x*(-0.5*b_{y_g} + 0.5*w_y) + q_y*(0.5*b_{x_g} - 0.5*w_x)) + q_z], [b_{x_g}], [b_{y_g}], [b_{z_g}], [b_{x_m}], [b_{y_m}], [b_{z_m}]])


In [10]:
state_transition_function_f

Matrix([
[  \Delta t*(q_x*(0.5*b_{x_g} - 0.5*w_x) + q_y*(0.5*b_{y_g} - 0.5*w_y) + q_z*(0.5*b_{z_g} - 0.5*w_z)) + q_w],
[\Delta t*(q_w*(-0.5*b_{x_g} + 0.5*w_x) + q_y*(-0.5*b_{z_g} + 0.5*w_z) + q_z*(0.5*b_{y_g} - 0.5*w_y)) + q_x],
[\Delta t*(q_w*(-0.5*b_{y_g} + 0.5*w_y) + q_x*(0.5*b_{z_g} - 0.5*w_z) + q_z*(-0.5*b_{x_g} + 0.5*w_x)) + q_y],
[\Delta t*(q_w*(-0.5*b_{z_g} + 0.5*w_z) + q_x*(-0.5*b_{y_g} + 0.5*w_y) + q_y*(0.5*b_{x_g} - 0.5*w_x)) + q_z],
[                                                                                                   b_{x_g}],
[                                                                                                   b_{y_g}],
[                                                                                                   b_{z_g}],
[                                                                                                   b_{x_m}],
[                                                                                                   b_{y_m}],
[

$$
s_{k+1} = As_k + Bu_k
$$

In [11]:
state_transition_matrix_A = state_transition_function_f.jacobian(states_s)
print(state_transition_matrix_A)

Matrix([[1, \Delta t*(0.5*b_{x_g} - 0.5*w_x), \Delta t*(0.5*b_{y_g} - 0.5*w_y), \Delta t*(0.5*b_{z_g} - 0.5*w_z), 0.5*\Delta t*q_x, 0.5*\Delta t*q_y, 0.5*\Delta t*q_z, 0, 0, 0], [\Delta t*(-0.5*b_{x_g} + 0.5*w_x), 1, \Delta t*(-0.5*b_{z_g} + 0.5*w_z), \Delta t*(0.5*b_{y_g} - 0.5*w_y), -0.5*\Delta t*q_w, 0.5*\Delta t*q_z, -0.5*\Delta t*q_y, 0, 0, 0], [\Delta t*(-0.5*b_{y_g} + 0.5*w_y), \Delta t*(0.5*b_{z_g} - 0.5*w_z), 1, \Delta t*(-0.5*b_{x_g} + 0.5*w_x), -0.5*\Delta t*q_z, -0.5*\Delta t*q_w, 0.5*\Delta t*q_x, 0, 0, 0], [\Delta t*(-0.5*b_{z_g} + 0.5*w_z), \Delta t*(-0.5*b_{y_g} + 0.5*w_y), \Delta t*(0.5*b_{x_g} - 0.5*w_x), 1, 0.5*\Delta t*q_y, -0.5*\Delta t*q_x, -0.5*\Delta t*q_w, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])


In [12]:
state_transition_matrix_A

Matrix([
[                                1,  \Delta t*(0.5*b_{x_g} - 0.5*w_x),  \Delta t*(0.5*b_{y_g} - 0.5*w_y),  \Delta t*(0.5*b_{z_g} - 0.5*w_z),  0.5*\Delta t*q_x,  0.5*\Delta t*q_y,  0.5*\Delta t*q_z, 0, 0, 0],
[\Delta t*(-0.5*b_{x_g} + 0.5*w_x),                                 1, \Delta t*(-0.5*b_{z_g} + 0.5*w_z),  \Delta t*(0.5*b_{y_g} - 0.5*w_y), -0.5*\Delta t*q_w,  0.5*\Delta t*q_z, -0.5*\Delta t*q_y, 0, 0, 0],
[\Delta t*(-0.5*b_{y_g} + 0.5*w_y),  \Delta t*(0.5*b_{z_g} - 0.5*w_z),                                 1, \Delta t*(-0.5*b_{x_g} + 0.5*w_x), -0.5*\Delta t*q_z, -0.5*\Delta t*q_w,  0.5*\Delta t*q_x, 0, 0, 0],
[\Delta t*(-0.5*b_{z_g} + 0.5*w_z), \Delta t*(-0.5*b_{y_g} + 0.5*w_y),  \Delta t*(0.5*b_{x_g} - 0.5*w_x),                                 1,  0.5*\Delta t*q_y, -0.5*\Delta t*q_x, -0.5*\Delta t*q_w, 0, 0, 0],
[                                0,                                 0,                                 0,                                 0,                 1,

In [13]:
input_matrix_B = state_transition_function_f.jacobian(inputs_u)

In [14]:
input_matrix_B

Matrix([
[-0.5*\Delta t*q_x, -0.5*\Delta t*q_y, -0.5*\Delta t*q_z],
[ 0.5*\Delta t*q_w, -0.5*\Delta t*q_z,  0.5*\Delta t*q_y],
[ 0.5*\Delta t*q_z,  0.5*\Delta t*q_w, -0.5*\Delta t*q_x],
[-0.5*\Delta t*q_y,  0.5*\Delta t*q_x,  0.5*\Delta t*q_w],
[                0,                 0,                 0],
[                0,                 0,                 0],
[                0,                 0,                 0],
[                0,                 0,                 0],
[                0,                 0,                 0],
[                0,                 0,                 0]])

The observation model ($h$) is defined a follows:
$$
y_k = h(s_k, u_k)
$$

In [15]:
observation_model_h = Matrix([
    rot_mat_from_quat(quaternion).T*(gravity_vec) + bias_accelerometer, # accelerometer
    rot_mat_from_quat(quaternion).T * mag_local + bias_magnetometer, # magnetometer
])

In [16]:
observation_model_h

Matrix([
[                                                                 b_{x_a} + g*(-2*q_w*q_y + 2*q_x*q_z)],
[                                                                  b_{y_a} + g*(2*q_w*q_x + 2*q_y*q_z)],
[                                                               b_{z_a} + g*(-2*q_x**2 - 2*q_y**2 + 1)],
[b_{x_m} + m_x*(-2*q_y**2 - 2*q_z**2 + 1) + m_y*(2*q_w*q_z + 2*q_x*q_y) + m_z*(-2*q_w*q_y + 2*q_x*q_z)],
[b_{y_m} + m_x*(-2*q_w*q_z + 2*q_x*q_y) + m_y*(-2*q_x**2 - 2*q_z**2 + 1) + m_z*(2*q_w*q_x + 2*q_y*q_z)],
[b_{z_m} + m_x*(2*q_w*q_y + 2*q_x*q_z) + m_y*(-2*q_w*q_x + 2*q_y*q_z) + m_z*(-2*q_x**2 - 2*q_y**2 + 1)]])

In [17]:
observation_matrix_H = observation_model_h.jacobian(states_s)

In [18]:
observation_matrix_H

Matrix([
[              -2*g*q_y,                           2*g*q_z,                           -2*g*q_w,                            2*g*q_x, 0, 0, 0, 0, 0, 0],
[               2*g*q_x,                           2*g*q_w,                            2*g*q_z,                            2*g*q_y, 0, 0, 0, 0, 0, 0],
[                     0,                          -4*g*q_x,                           -4*g*q_y,                                  0, 0, 0, 0, 0, 0, 0],
[ 2*m_y*q_z - 2*m_z*q_y,             2*m_y*q_y + 2*m_z*q_z, -4*m_x*q_y + 2*m_y*q_x - 2*m_z*q_w, -4*m_x*q_z + 2*m_y*q_w + 2*m_z*q_x, 0, 0, 0, 1, 0, 0],
[-2*m_x*q_z + 2*m_z*q_x, 2*m_x*q_y - 4*m_y*q_x + 2*m_z*q_w,              2*m_x*q_x + 2*m_z*q_z, -2*m_x*q_w - 4*m_y*q_z + 2*m_z*q_y, 0, 0, 0, 0, 1, 0],
[ 2*m_x*q_y - 2*m_y*q_x, 2*m_x*q_z - 2*m_y*q_w - 4*m_z*q_x,  2*m_x*q_w + 2*m_y*q_z - 4*m_z*q_y,              2*m_x*q_x + 2*m_y*q_y, 0, 0, 0, 0, 0, 1]])

In [19]:
# Substitute equilibrium point values:
# Equilibrium quaternion (slight rotation)
# Zero biases and slight gyro
subs_dict = {
    qw: 0.9862359, qx: 0.0544469, qy: 0.0806561, qz: 0.1336749,
    # qw: 1, qx: 0, qy: 0, qz: 0,  
    bx_g: 0, by_g: 0, bz_g: 0,
    bx_a: 0, by_a: 0, bz_a: 0,
    bx_m: 0, by_m: 0, bz_m: 0,
    wx: 0.1, wy: 0.1, wz: 0.1,
    dt: 0.1,
    g: 9.81,
    mag_x: 0.3, mag_y: 0.0, mag_z: 0.5
}

A_num = state_transition_matrix_A.subs(subs_dict).evalf()
B_num = input_matrix_B.subs(subs_dict).evalf()
H_num = observation_matrix_H.subs(subs_dict).evalf()

# Convert sympy matrices to numpy arrays for observability calculation
A_np = np.array(A_num.tolist(), dtype=np.float64)
H_np = np.array(H_num.tolist(), dtype=np.float64)

In [20]:
A_num

Matrix([
[  1.0, -0.005, -0.005, -0.005,  0.002722345,  0.004032805,  0.006683745,   0,   0,   0],
[0.005,    1.0,  0.005, -0.005, -0.049311795,  0.006683745, -0.004032805,   0,   0,   0],
[0.005, -0.005,    1.0,  0.005, -0.006683745, -0.049311795,  0.002722345,   0,   0,   0],
[0.005,  0.005, -0.005,    1.0,  0.004032805, -0.002722345, -0.049311795,   0,   0,   0],
[    0,      0,      0,      0,          1.0,            0,            0,   0,   0,   0],
[    0,      0,      0,      0,            0,          1.0,            0,   0,   0,   0],
[    0,      0,      0,      0,            0,            0,          1.0,   0,   0,   0],
[    0,      0,      0,      0,            0,            0,            0, 1.0,   0,   0],
[    0,      0,      0,      0,            0,            0,            0,   0, 1.0,   0],
[    0,      0,      0,      0,            0,            0,            0,   0,   0, 1.0]])

In [21]:
B_num

Matrix([
[-0.002722345, -0.004032805, -0.006683745],
[ 0.049311795, -0.006683745,  0.004032805],
[ 0.006683745,  0.049311795, -0.002722345],
[-0.004032805,  0.002722345,  0.049311795],
[           0,            0,            0],
[           0,            0,            0],
[           0,            0,            0],
[           0,            0,            0],
[           0,            0,            0],
[           0,            0,            0]])

In [22]:
H_num

Matrix([
[-1.582472682,  2.622701538, -19.349948358, 1.068248178, 0, 0, 0,   0,   0,   0],
[ 1.068248178, 19.349948358,   2.622701538, 1.582472682, 0, 0, 0,   0,   0,   0],
[           0, -2.136496356,  -3.164945364,           0, 0, 0, 0,   0,   0,   0],
[  -0.0806561,    0.1336749,   -1.08302322, -0.10596298, 0, 0, 0, 1.0,   0,   0],
[ -0.02575804,   1.03462956,    0.16634304, -0.51108544, 0, 0, 0,   0, 1.0,   0],
[  0.04839366,  -0.02868886,    0.43042934,  0.03266814, 0, 0, 0,   0,   0, 1.0]])

In [23]:
# Build observability matrix [H; H A; H A^2; ... H A^(n-1)]
n_states = A_np.shape[0]
obs_matrix = H_np.copy()
for i in range(1, n_states):
    obs_matrix = np.vstack((obs_matrix, H_np @ np.linalg.matrix_power(A_np, i)))

rank_obs = np.linalg.matrix_rank(obs_matrix)

print(f"\nObservability matrix rank: {rank_obs} out of {n_states} states")

if rank_obs == n_states:
    print("The system is fully observable at the equilibrium point.")
else:
    print("The system is NOT fully observable at the equilibrium point.")


Observability matrix rank: 10 out of 10 states
The system is fully observable at the equilibrium point.


In [24]:
Matrix(obs_matrix.tolist())

Matrix([
[        -1.582472682,         2.622701538,     -19.349948358,         1.068248178,                     0,                     0,                     0,   0,   0,   0],
[         1.068248178,        19.349948358,       2.622701538,         1.582472682,                     0,                     0,                     0,   0,   0,   0],
[                   0,        -2.136496356,      -3.164945364,                   0,                     0,                     0,                     0,   0,   0,   0],
[          -0.0806561,           0.1336749,       -1.08302322,         -0.10596298,                     0,                     0,                     0, 1.0,   0,   0],
[         -0.02575804,          1.03462956,        0.16634304,         -0.51108544,                     0,                     0,                     0,   0, 1.0,   0],
[          0.04839366,         -0.02868886,        0.43042934,          0.03266814,                     0,                     0,                 

In [25]:
from scipy.linalg import null_space

nullsp = null_space(obs_matrix)
if nullsp.shape[1] > 0:
    print("Basis vectors for unobservable states:")
    for i in range(nullsp.shape[1]):
        print(np.round(nullsp[:, i]))
        unobserved_states = [state for state, include in zip(states_s, abs(np.round(nullsp[:, i])) > 0.1) if include]
        print(f"Which corresponds to: {unobserved_states}")

In [26]:
process_noise_covariance_Q = np.diag([
    1.0e-6, # qw
    1.0e-6, # qx
    1.0e-6, # qy
    1.0e-6, # qz
    1.0e-6, # gyro bias (x)
    1.0e-6, # gyro bias (y)
    1.0e-6, # gyro bias (z)
    1.0e-6, # magnetometer bias (x)
    1.0e-6, # magnetometer bias (y)
    1.0e-6, # magnetometer bias (z)
])
measurement_noise_covariance_R = np.diag([
    1.0e2, # accelerometer (x)
    1.0e2, # accelerometer (y)
    1.0e2, # accelerometer (z)
    1.0e2, # magnetometer (x)
    1.0e2, # magnetometer (y)
    1.0e2, # magnetometer (z)
])

estimation_covariance_P = np.eye(n_states)
estimation_covariance_P = (
    A_np @ estimation_covariance_P @ A_np.T + process_noise_covariance_Q
)

kalman_gain_K = (
    estimation_covariance_P @ H_np.T @ np.linalg.inv(
        H_np @ estimation_covariance_P @ H_np.T + measurement_noise_covariance_R
    )
)
print(kalman_gain_K)

[[-3.56767608e-03  2.50805591e-03  2.82198367e-03 -1.14336218e-04
  -6.93359634e-04  1.98066172e-04]
 [ 5.90328167e-03  3.93694093e-02 -4.41867919e-03  4.07832066e-04
   2.50799654e-03 -1.09960255e-04]
 [-3.90583115e-02  4.73125991e-03 -6.54654164e-03 -2.22539690e-03
   1.58096684e-04  8.69679613e-04]
 [ 2.22702795e-03  3.52661855e-03  3.59868058e-04 -1.49503464e-03
  -5.65153837e-03  4.85000692e-04]
 [-3.06978191e-05 -1.94693351e-03  2.70086365e-04 -1.15476449e-05
  -1.49025715e-04  2.09933056e-06]
 [ 1.94004345e-03  3.02640413e-05  3.02908926e-04  1.15774830e-04
   2.15006455e-05 -4.40285684e-05]
 [-2.63123360e-04 -3.02251115e-04  1.11054516e-06  6.50879756e-05
   2.63690130e-04 -1.97305887e-05]
 [-4.27148455e-04  4.33013814e-06 -6.17193405e-05  9.87493736e-03
  -8.10705099e-06  1.01381239e-05]
 [ 1.42146059e-05 -3.92601980e-04  5.85869870e-05 -8.10705099e-06
   9.84627288e-03  2.19882958e-06]
 [ 1.69119986e-04 -1.13227012e-05  2.51756129e-05  1.01381239e-05
   2.19882958e-06  9.8970